In [1]:
import cv2
import os
import imageio
import numpy as np
from keras.models import load_model

In [2]:
path = "C:\\Users\\nikhil\\Downloads\\archive\\img_align_celeba\\img_align_celeba\\"
images = os.listdir(path)

In [3]:
for i in range(35001, 70001):
    image = imageio.imread(path+images[i])
    imageio.imwrite("./images/"+images[i], image)

In [4]:
import pandas as pd
from tqdm import tqdm

In [5]:
def get_data_from_local(file, l, b):
    face = cv2.imread(file,cv2.IMREAD_COLOR)
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face =cv2.resize(face, (l, b))
    face = np.asarray(face)
    face = face.astype('float32')
    face /= 255
    return np.array(face)

In [9]:
d = "images/"
real_images_path = []
# for path in os.listdir(d):
#     full_path = os.path.join(d, path)
#     if os.path.isfile(full_path):
#         real_images_path.append(str(full_path))
for i in range(35001, 70002):
    full_path = os.path.join(d, "0" + str(i) + ".jpg")
    if os.path.isfile(full_path):
        real_images_path.append(str(full_path))

In [10]:
len(real_images_path)

35001

In [11]:
data = {
    'age':[],
    'gender':[],
    'race':[],
    'eye_color':[],
    'hair_color':[],
    'beard': [],
    'face_color':[],
    'image':[]
}
age_model = load_model('model/age_model_checkpoint.tf')
gender_model = load_model('model/gender_model_checkpoint.tf')
race_model = load_model('model/race_model_checkpoint.tf')
eye_color_model = load_model('model/eye_color_model_checkpoint.tf')
hair_color_model = load_model('model/hair_color_model_checkpoint.tf')
beard_model = load_model('model/beard_model_checkpoint.tf')

for image in real_images_path:
    temp = image
    image_AGR = get_data_from_local(image, 200, 200)
    image_AGR = np.expand_dims(image_AGR, axis=0)
    
    image_eye_color = get_data_from_local(image, 300, 300)
    image_eye_color = np.expand_dims(image_eye_color, axis=0)
    
    image_hCB = get_data_from_local(image, 178, 218)
    image_hCB = np.expand_dims(image_hCB, axis=0)
    
    data['age'].append(age_model.predict(image_AGR).argmax())
    data['gender'].append(gender_model.predict(image_AGR).argmax())
    data['race'].append(race_model.predict(image_AGR).argmax())
    data['eye_color'].append(eye_color_model.predict(image_eye_color).argmax())
    data['hair_color'].append(hair_color_model.predict(image_hCB).argmax())
    data['beard'].append(beard_model.predict(image_hCB).argmax())
    data['image'].append(temp)

In [12]:
dataframe = pd.DataFrame(data,columns=['age','gender','race','eye_color','hair_color','beard','image'])

In [13]:
dataframe.to_csv('main_dataset1.csv')